In [1]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from textblob import TextBlob
from wordcloud import WordCloud

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re

In [5]:
df = pd.read_csv("flipkart_product.csv",encoding="latin-1")

In [6]:
df.head()

,ProductName,Price,Rate,Review,Summary
0,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Super!,Great cooler.. excellent air flow and for this...
1,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",5,Awesome,Best budget 2 fit cooler. Nice cooling
2,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,The quality is good but the power of air is de...
3,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",1,Useless product,Very bad product it's a only a fan
4,Candes 12 L Room/Personal Air Cooler?ÿ?ÿ(White...,"??3,999",3,Fair,Ok ok product


In [7]:
def remove_punc(str):
    return re.sub(r"[^\w\s]","",str)

In [8]:
def remove_stopwords(input_str):
    # Get the set of English stopwords
    stop_words = set(stopwords.words("english"))
    
    # Convert input string to lowercase and split into words
    words = input_str.lower().split()
    
    # Filter out stopwords
    filtered_words = [w for w in words if w not in stop_words]
    
    # Join the filtered words into a single string
    return " ".join(filtered_words)

In [9]:
def clean_text(text):
    # Remove non-ASCII characters
    text = ''.join([i if ord(i) < 128 else ' ' for i in text])
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase and split text into words
    words = text.lower().split()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # Join the cleaned words back into a single string
    return ' '.join(filtered_words)

In [10]:
df["ProductName"] = df["ProductName"].astype(str)
df["ProductName"] = df["ProductName"].apply(remove_punc)
df["ProductName"] = df["ProductName"].apply(remove_stopwords)
#df[["ProductName"]].head(7)

df["Review"] = df["Review"].astype(str)
df["Review"] = df["Review"].apply(remove_punc)
df["Review"] = df["Review"].apply(remove_stopwords)

df["Summary"] = df["Summary"].astype(str).apply(remove_punc)
df["Summary"] = df["Summary"].apply(remove_stopwords)

In [11]:
def clean_price(price):
    if pd.isna(price):  # Handle NaN values
        return np.nan
    # Extract numbers from the price
    price = re.sub(r'[^\d]', '', price)
    return int(price) if price else np.nan

df["Price"] = df["Price"].apply(clean_price)
df["Price"][189873]=142
df["Rate"][189873] = 4

C:\Users\hp\AppData\Local\Temp\ipykernel_2500\1883408646.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Price"][189873]=142
C:\Users\hp\AppData\Local\Temp\ipykernel_2500\1883408646.py:9: SettingWithCopyWarning: 
A value is trying to

In [12]:
df.dtypes

ProductName     object
Price          float64
Rate            object
Review          object
Summary         object
dtype: object

In [13]:
df.isnull().sum()

ProductName    0
Price          0
Rate           0
Review         0
Summary        0
dtype: int64

In [14]:
df["Rate"].unique()

array(['5', '3', '1', '4', '2',
       'Pigeon Favourite Electric Kettle?ÿ?ÿ(1.5 L, Silver, Black)',
       'Bajaj DX 2 L/W Dry Iron',
       'Nova Plus Amaze NI 10 1100 W Dry Iron?ÿ?ÿ(Grey & Turquoise)', 's',
       4], dtype=object)

In [15]:
df["Rate"] = df["Rate"].astype(str)
df["Rate"] = df["Rate"].apply(clean_text)
df["Rate"] = df["Rate"].apply(remove_punc)
df["Rate"] = df["Rate"].apply(remove_stopwords)

In [16]:
df["Rate"].unique()

array(['5', '3', '1', '4', '2',
       'pigeon favourite electric kettle 15 l silver black',
       'bajaj dx 2 lw dry iron',
       'nova plus amaze ni 10 1100 w dry iron grey turquoise', ''],
      dtype=object)

In [17]:
df["ProductName"] = df["ProductName"].astype(str)
df["ProductName"] = df["ProductName"].apply(clean_text)

In [18]:
df_Summary = df.drop(["ProductName","Price","Rate","Review"],axis=1)
print("Dataframe summary:\n")
print(df_Summary)

Dataframe summary:

                                                  Summary
0       great cooler excellent air flow price amazing ...
1                   best budget 2 fit cooler nice cooling
2                           quality good power air decent
3                                         bad product fan
4                                           ok ok product
...                                                   ...
189869                                             thanks
189870                                               good
189871                                              super
189872                                               good
189873                                                nan

[189874 rows x 1 columns]


In [19]:
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))
def data_preprocessing(text):
    text = text.lower()
    text = re.sub(r"https:\S+|www\S+https\S+", '',text,flags=re.MULTILINE)
    text = re.sub(r"\@\w+|\#", '',text)
    text = re.sub(r'[^\w\s]', '',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return  " ".join(filtered_text) 

df["Proc_summary"] = df_Summary["Summary"].apply(data_preprocessing)
df["Proc_summary"]

0         great cooler excellent air flow price amazing ...
1                     best budget 2 fit cooler nice cooling
2                             quality good power air decent
3                                           bad product fan
4                                             ok ok product
                                ...                        
189869                                               thanks
189870                                                 good
189871                                                super
189872                                                 good
189873                                                  nan
Name: Proc_summary, Length: 189874, dtype: object

In [20]:
df.head()

,ProductName,Price,Rate,Review,Summary,Proc_summary
0,candes 12 l roompersonal air cooler white blac...,3999.0,5,super,great cooler excellent air flow price amazing ...,great cooler excellent air flow price amazing ...
1,candes 12 l roompersonal air cooler white blac...,3999.0,5,awesome,best budget 2 fit cooler nice cooling,best budget 2 fit cooler nice cooling
2,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,quality good power air decent,quality good power air decent
3,candes 12 l roompersonal air cooler white blac...,3999.0,1,useless product,bad product fan,bad product fan
4,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,ok ok product,ok ok product


In [21]:
from textblob import TextBlob
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return "positive"
    elif polarity < 0:
        return "negative"
    else:
        return "neutral"

In [22]:
df["Sentiment"] = df["Review"].apply(get_sentiment)
df.head()

,ProductName,Price,Rate,Review,Summary,Proc_summary,Sentiment
0,candes 12 l roompersonal air cooler white blac...,3999.0,5,super,great cooler excellent air flow price amazing ...,great cooler excellent air flow price amazing ...,positive
1,candes 12 l roompersonal air cooler white blac...,3999.0,5,awesome,best budget 2 fit cooler nice cooling,best budget 2 fit cooler nice cooling,positive
2,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,quality good power air decent,quality good power air decent,positive
3,candes 12 l roompersonal air cooler white blac...,3999.0,1,useless product,bad product fan,bad product fan,negative
4,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,ok ok product,ok ok product,positive


In [23]:
df["Sentiment"].value_counts()

Sentiment
positive    123371
neutral      52750
negative     13753
Name: count, dtype: int64

In [24]:
df["Sentiment_val"] = df["Sentiment"].map({"positive":1,"negative":-1,"neutral":0})
df.head()

,ProductName,Price,Rate,Review,Summary,Proc_summary,Sentiment,Sentiment_val
0,candes 12 l roompersonal air cooler white blac...,3999.0,5,super,great cooler excellent air flow price amazing ...,great cooler excellent air flow price amazing ...,positive,1
1,candes 12 l roompersonal air cooler white blac...,3999.0,5,awesome,best budget 2 fit cooler nice cooling,best budget 2 fit cooler nice cooling,positive,1
2,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,quality good power air decent,quality good power air decent,positive,1
3,candes 12 l roompersonal air cooler white blac...,3999.0,1,useless product,bad product fan,bad product fan,negative,-1
4,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,ok ok product,ok ok product,positive,1


In [25]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.naive_bayes import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score
from sklearn.neighbors import *

In [33]:
df["Combined_s"] = df["Summary"] + df['Review']

In [34]:
df.head()

,ProductName,Price,Rate,Review,Summary,Proc_summary,Sentiment,Sentiment_val,Combined_s
0,candes 12 l roompersonal air cooler white blac...,3999.0,5,super,great cooler excellent air flow price amazing ...,great cooler excellent air flow price amazing ...,positive,1,great cooler excellent air flow price amazing ...
1,candes 12 l roompersonal air cooler white blac...,3999.0,5,awesome,best budget 2 fit cooler nice cooling,best budget 2 fit cooler nice cooling,positive,1,best budget 2 fit cooler nice coolingawesome
2,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,quality good power air decent,quality good power air decent,positive,1,quality good power air decentfair
3,candes 12 l roompersonal air cooler white blac...,3999.0,1,useless product,bad product fan,bad product fan,negative,-1,bad product fanuseless product
4,candes 12 l roompersonal air cooler white blac...,3999.0,3,fair,ok ok product,ok ok product,positive,1,ok ok productfair


In [35]:
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features for performance
X = vectorizer.fit_transform(df['Combined_s'])

In [36]:
y = df['Sentiment_val']
np.array(y)

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
lr_model = LogisticRegression(max_iter=1000,multi_class="multinomial")
lr_model.fit(X_train, y_train)

c:\Users\hp\Desktop\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [39]:
lgr_pred = lr_model.predict(X_test)

In [40]:
accuracy_score(lgr_pred, y_test)

0.9295589203423305

In [41]:
rfc = RandomForestClassifier(n_estimators=100,max_depth=100)
rfc.fit(X_train,y_train)

RandomForestClassifier(max_depth=100)

In [42]:
rfc_pred = rfc.predict(X_test)

In [43]:
accuracy_score(rfc_pred, y_test)

0.888953258722844

In [45]:
confusion_matrix(rfc_pred, y_test)

array([[ 1709,   119,    57],
       [   95,  7534,   222],
       [  904,  2820, 24515]], dtype=int64)

In [47]:
print(classification_report(rfc_pred, y_test))

              precision    recall  f1-score   support

          -1       0.63      0.91      0.74      1885
           0       0.72      0.96      0.82      7851
           1       0.99      0.87      0.92     28239

    accuracy                           0.89     37975
   macro avg       0.78      0.91      0.83     37975
weighted avg       0.92      0.89      0.89     37975



In [48]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=20)

In [49]:
knn_pred = knn.predict(X_test)

In [50]:
accuracy_score(knn_pred, y_test)

0.8771560236998025

In [51]:
estimators = [('lr', lr_model), ('knn', knn), ('rfc', rfc)]
stack_model = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier(max_depth=400))
stack_model.fit(X_train, y_train)

c:\Users\hp\Desktop\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\hp\Desktop\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\hp\Desktop\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\hp\Desktop\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1247: Futur

StackingClassifier(estimators=[('lr',
                                LogisticRegression(max_iter=1000,
                                                   multi_class='multinomial')),
                               ('knn', KNeighborsClassifier(n_neighbors=20)),
                               ('rfc', RandomForestClassifier(max_depth=100))],
                   final_estimator=DecisionTreeClassifier(max_depth=400))

In [52]:
stack_pred = stack_model.predict(X_test)

In [53]:
accuracy_score(stack_pred, y_test)

0.9037524687294273

In [54]:
confusion_matrix(stack_pred, y_test)

array([[ 2095,   275,   393],
       [  236,  8852,  1028],
       [  377,  1346, 23373]], dtype=int64)

#ACCEPTED